In [1]:
import pandas as pd

### Leitura dados posse

In [127]:
sheet_id = '1SsVygtWx_Tv6Ps5zjhxeNJcDincPk6jpse-O5aFHQWY'
sheet_name = 'DadosPosseTJ'
url = f'https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}'
df_posse = pd.read_csv(url)

C:\Users\labci\AppData\Local\Temp\ipykernel_12676\979523307.py:4: DtypeWarning: Columns (36,37) have mixed types. Specify dtype option on import or set low_memory=False.
  df_posse = pd.read_csv(url)


In [128]:
# Filtrar por imobiliário
df_posse = df_posse.query('Imobiliário == "Sim"')
len(df_posse)

55015

In [129]:
# Filtrar para RMSP (1a RAJ + )
df_posse = df_posse.query('UTA in @cod_utas')
len(df_posse)

21737

In [130]:
# Carrega dados das UTAs
utas = pd.read_csv('wkt_utas_rmsp.txt', sep='\t', usecols=['COD_UTA', 'DOM_PP'])
utas = utas.rename(columns={'COD_UTA':'UTA'})
cod_utas = utas['UTA'].to_list()

In [131]:
def classificar_sentença(sentenca):
    if not sentenca or not isinstance(sentenca, str):
        return None
        
    sentencas = list(set([i.strip() for i in sentenca.split(',')]))
    if 'Extinção' in sentencas:
        return 'Extinta'
    if 'Procedente' in sentencas and 'Improcedente' not in sentencas:
        return 'Procedente'
    
    elif 'Improcedente' in sentencas and 'Procedente' in sentencas:
        return 'Múltipla'

    elif 'Improcedente' in sentencas:
        return 'Improcedente'

    elif 'Homologação' in sentencas:
        return 'Homologação'

    else:
        return 'Sentença não identificada'

In [132]:
# Calcular novas colunas
# Calculo do ano
df_posse['Ano'] = df_posse['Data de Disponibilização'].apply(lambda x: int(x[-4:]))

# Calculo de Ano-Mês
df_posse['Ano-Mês'] = df_posse['Data de Disponibilização'].apply(lambda x: x.split('/')[-1] + '-' + x.split('/')[-2])

# Calculo da sentença
df_posse['Desfecho'] = df_posse['Sentença'].apply(classificar_sentença)

In [133]:
# Linha do tempo
export = df_posse.query('Ano >= 2017').pivot_table(index='Ano-Mês', values='Processo', aggfunc='count').reset_index()
export = export.rename(columns = {'Processo':'Processos'})

export.to_csv('posse_por_ano-mes.csv', sep=';', index=False)

In [189]:
# Mapa dos conflitos de posse RMSP
recorte = ['Procedente', 'Homologação']
export = df_posse.query('Desfecho in @recorte').pivot_table(index='UTA', values='Processo', aggfunc='count').reset_index()
export = export.merge(utas, on='UTA')
export['Processos por domicilio'] = round(export['Processo']*100000/export['DOM_PP'],0).astype(int)
export = export.rename(columns={'Processo':'Processos'})
export = export[['UTA', 'Processos', 'Processos por domicilio']]

export.to_csv('posse_por_uta.csv', sep=';', index=False)

In [136]:
# Desfecho por ano
export = df_posse.query('Ano >= 2017').pivot_table(index='Desfecho', values='Processo', columns='Ano', aggfunc='count').reset_index()
export.to_csv('posse_desfecho_por_ano.csv', sep=';', index=False)

In [171]:
# Ano etiqueta versus ano da sentença
export = df_posse.query('`Ano etiqueta` > 2013')\
    .query('Desfecho in @recorte')\
    .pivot_table(columns='Ano etiqueta' , index = 'Ano', values='Processo', aggfunc='count')\
    .fillna(0).reset_index()
#export['Ano etiqueta'] = export['Ano etiqueta'].apply(lambda x: 'Abertura em ' + str(x))
export = export.rename(columns={i:'Abertura em '+ str(i) for i in export.columns if isinstance(i, int)}) #= export.melt(id_vars = ['Ano etiqueta'])

In [172]:
export.to_csv('posse_etiqueta_vs_datasentenca.csv', sep=';', index=False)

### Leitura dados despejos

In [201]:
refs_despejos = {
    '14Wk3pH9EfSbf2zXQ2D0v_aAqxG-NCCoR': 'Desp2018',
    '12i2EntCW8txx5pwYuf05GZS1993GrLa1': 'Desp2019',
    '1oX3N-PI92jmsMdxlsuHW58_0MLuhfvuL': 'Desp2020',
    '1oQtwBBV12kVq0wshJFA-OySJaFauUKEvyFkRez_zbdA': 'Desp2021',
    '1mTSGVHTbMD_GUi9Y9GFu7StfqVRO6Rmym0hyljSv7-g': 'Desp2022',
}

df_aluguel = pd.DataFrame()

for sheet_id, sheet_name in refs_despejos.items():
    print(sheet_name)
    url = f'https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}'
    df_temp = pd.read_csv(url)
    df_aluguel = pd.concat([df_aluguel, df_temp])

Desp2018


C:\Users\labci\AppData\Local\Temp\ipykernel_12676\1412548370.py:14: DtypeWarning: Columns (40) have mixed types. Specify dtype option on import or set low_memory=False.
  df_temp = pd.read_csv(url)


Desp2019
Desp2020


C:\Users\labci\AppData\Local\Temp\ipykernel_12676\1412548370.py:14: DtypeWarning: Columns (36) have mixed types. Specify dtype option on import or set low_memory=False.
  df_temp = pd.read_csv(url)


Desp2021


C:\Users\labci\AppData\Local\Temp\ipykernel_12676\1412548370.py:14: DtypeWarning: Columns (29) have mixed types. Specify dtype option on import or set low_memory=False.
  df_temp = pd.read_csv(url)


Desp2022


C:\Users\labci\AppData\Local\Temp\ipykernel_12676\1412548370.py:14: DtypeWarning: Columns (29,40) have mixed types. Specify dtype option on import or set low_memory=False.
  df_temp = pd.read_csv(url)


In [202]:
# Filtrar para RMSP (1a RAJ + )
df_aluguel = df_aluguel.query('UTA in @cod_utas')
len(df_aluguel)

69188

In [203]:
# Calcular novas colunas
# Calculo do ano
df_aluguel['Ano'] = df_aluguel['Data de Disponibilização'].apply(lambda x: int(x[-4:]))

# Calculo de Ano-Mês
df_aluguel['Ano-Mês'] = df_aluguel['Data de Disponibilização'].apply(lambda x: x.split('/')[-1] + '-' + x.split('/')[-2])

# Calculo da sentença
df_aluguel['Desfecho'] = df_aluguel['Sentença'].apply(classificar_sentença)

In [204]:
# Mapa dos conflitos de aluguel
export = df_aluguel.query('Desfecho in @recorte').pivot_table(index='UTA', values='Processo', aggfunc='count').reset_index()
export = export.merge(utas, on='UTA')
export['Processos por domicilio'] = round(export['Processo']*100000/export['DOM_PP'],0).astype(int)
export = export.rename(columns={'Processo':'Processos'})
export = export[['UTA', 'Processos', 'Processos por domicilio']]

export.to_csv('aluguel_por_uta.csv', sep=';', index=False)

In [205]:
# Linha do tempo
export = df_aluguel.query('Ano >= 2017').pivot_table(index='Ano-Mês', values='Processo', aggfunc='count').reset_index()
export = export.rename(columns = {'Processo':'Processos'})

export.to_csv('aluguel_por_ano-mes.csv', sep=';', index=False)

In [206]:
# Desfecho por ano
export = df_aluguel.query('Ano >= 2017').pivot_table(index='Desfecho', values='Processo', columns='Ano', aggfunc='count').reset_index()
export.to_csv('aluguel_desfecho_por_ano.csv', sep=';', index=False)

In [212]:
df_aluguel['Mês'] = df_aluguel['Data de Disponibilização'].apply(lambda x: x.split('/')[1])

In [217]:
# Sentenças procedentes de aluguel por ano
export = df_aluguel.query('Ano >= 2017')\
    .query('Desfecho in @recorte')\
    .pivot_table(index='Mês', columns='Ano', values='Processo', aggfunc='count')\
    .reset_index()

export.to_csv('aluguel_desfecho_por_mes.csv', sep=';', index=False)